In [1]:
################################
# import all dependent libraries
################################
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import requests
from pyspark.sql.functions import explode, split, col, first
import time
from pyspark.sql.functions import udf, desc,asc,sum as Fsum
from pyspark.sql.types import (StringType, IntegerType, 
                               ArrayType,StructField, StructType)
# import datetime
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# # agg() functions
# from pyspark.sql.functions import (last,  first, max as Fmax,
#                                    count, countDistinct,
#                                    sum as Fsum,sumDistinct,
#                                    collect_list)
# from pyspark.sql.window import Window
# from pyspark.sql.functions import row_number

########################################################################
# Set up a SparkSession in local mode to process datasets in a data lake
########################################################################
spark = SparkSession \
    .builder \
    .appName("ABS Census 2016 Data Analytics") \
    .config('spark.ui.port',3000) \
    .getOrCreate()
#spark.sparkContext.getConf().getAll()  # check sparksession full info

###################################################################################
###################################################################################
# Q2. Which birthCountry groups are most represented in top half "wealth" echelon ? 
###################################################################################
columns = StructType([
  StructField("abs_datasets", StringType(), True),
  StructField("POA", StringType(), True),
  StructField("birthCountry", StringType(), True),    
  StructField("wealthGroup", StringType(), True), 
  StructField("count", IntegerType(), True)    
])

Dff = spark.read\
    .schema(columns)\
    .option("mode","DROPMALFORMED")\
    .csv("WIDXbcPOA2016.csv")
Dff=Dff.withColumn("postcode",split(col("POA"),",").getItem(0))\
         .withColumn("state",split(col("POA"),",").getItem(1))\
         .drop("POA","abs_datasets").na.drop() #dropna()
Dff=Dff.filter(col("count") != 0)

DffB1=Dff.filter(col("wealthGroup")=="$1,500-$1,749 ($78,000-$90,999)")
DffB1=DffB1.withColumn("band1",col("count")*1.625)
DffB1=DffB1.drop("count").groupBy("birthCountry").sum()
DffB2=Dff.filter(col("wealthGroup")=="$1,750-$1,999 ($91,000-$103,999)")
DffB2=DffB2.withColumn("band2",col("count")*1.875)
DffB2=DffB2.drop("count").groupBy("birthCountry").sum()
DffB3=Dff.filter(col("wealthGroup")=="$2,000-$2,999 ($104,000-$155,999)")
DffB3=DffB3.withColumn("band3",col("count")*2.5)
DffB3=DffB3.drop("count").groupBy("birthCountry").sum()
DffB4=Dff.filter(col("wealthGroup")=="$3,000 or more ($156,000 or more)")
DffB4=Dff.withColumn("band4",col("count")*3.5)
DffB4=DffB4.drop("count").groupBy("birthCountry").sum()
Dfft1=DffB1.join(DffB2,["birthCountry"],how='outer')#.distinct()
Dfft2=DffB3.join(DffB4,["birthCountry"],how='outer')#.distinct()
Dfft3=Dfft1.join(Dfft2,["birthCountry"],how='outer')#.distinct()
DffQ2=Dfft3.withColumn("wealthIndex from Census 2016",(col("sum(band1)")+col("sum(band2)")+\
                      col("sum(band3)")+col("sum(band4)"))/1000)
DffQ2.orderBy(desc("wealthIndex from Census 2016"))\
     .drop("sum(band1)","sum(band2)","sum(band3)","sum(band4)")\
     .show(truncate=False)

+--------------------------------+----------------------------+
|birthCountry                    |wealthIndex from Census 2016|
+--------------------------------+----------------------------+
|Australia                       |18132.608375                |
|New Zealand                     |848.280125                  |
|India                           |736.675625                  |
|China (excludes SARs and Taiwan)|403.520125                  |
|South Africa                    |400.01775                   |
|Malaysia                        |265.459125                  |
|Philippines                     |254.28875                   |
|Ireland                         |188.12775                   |
|Sri Lanka                       |172.882625                  |
|Vietnam                         |164.356625                  |
|United States of America        |162.6175                    |
|Hong Kong (SAR of China)        |147.59675                   |
|Germany                         |120.27